# 1. Ligando o Twitter

In [ ]:
%load_ext autoreload
%autoreload 2

#Inicializando o twitter bot
import tweepy
import settings

# Acrescente os 4 valores encontrados nas configurações do seu 
# app no Twitter
consumer_key = settings.CONSUMER_KEY
consumer_secret = settings.CONSUMER_SECRET
access_token = settings.ACCESS_TOKEN
access_token_secret = settings.ACCESS_TOKEN_SECRET

# Inicializando o Twitter usando Tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
bot = tweepy.API(auth)

In [ ]:
# testando o tweepy:
bot.get_user('burgos')

In [ ]:
# Mais legível:
burgos = bot.get_user('burgos')._json
burgos

In [ ]:
# Escrevendo uma mensagem:
bot.update_status('Testando!')

In [ ]:
# Deixando a imagem mais animada, usando esse dicionário:
# https://www.webpagefx.com/tools/emoji-cheat-sheet/
import emoji
emoji.emojize(":robot_face:")

# 2. Fazendo uma busca no Twitter

In [ ]:
# Criando uma busca:
busca = bot.search('#CodaBr18')

In [ ]:
# Exemplo de um tweet encontrado na busca:
busca[0]._json

In [ ]:
# Quais dados do tweet queremos usar?
def limpar_tweet(tweet_bruto):
    data = tweet_bruto._json
    status_id = data['id']
    user = data['user']['screen_name']
    return {"status_id": status_id, "user": user}

In [ ]:
limpar_tweet(busca[0])

## 🎶 Interlúdio sobre bots nas eleições 🎶

In [ ]:
# Fazendo scrape de uma lista de hashtags bombando na lista
from bs4 import BeautifulSoup as bs
import requests

soup = bs(requests.get('https://trends24.in/brazil/').content, "html.parser")
hashtags = [x.text for x in soup.find("ol", {"class": "trend-card__list"}).find_all("li")]
hashtags

In [ ]:
# Criamos um textinho com as principais hashtags
hashtags = (' ').join([hashtag for hashtag in hashtags[:4]])

In [ ]:
# Criamos uma lista de possíveis mensagens
mensagens_possiveis = [
    "Isso é mentira! Vote em Pedro Burgos 2022",
    "O Brasil precisa de Pedro Burgos agora"
]

In [ ]:
# Selecionamos uma aleatória
import random
msg = random.choice(mensagens_possiveis)

In [ ]:
# Criamos a mensagem
tweet = f'{msg} {hashtags}'

In [ ]:
# Tuitamos
bot.update_status(tweet)

# 3. Ligando todas as coisas

### Passos:
1. Buscar por alguma hashtag específica no Twitter  
2. Selecionar o principal tweet  
3. Tuitar para a pessoas  

Vamos criar funções!

In [ ]:
# Com o bot do Twitter já inicializado, Criamos duas funções

# Melhoramos a função de limpar tweets, para ter mais informações 
# dos usuários
def limpar_tweet(tweet_bruto):
    data = tweet_bruto._json
    status_id = data['id']
    user = data['user']['screen_name']
    rts = data['retweet_count']
    favs = data['favorite_count']
    followers = data['user']['followers_count']
    text = data['text']
    retweet = False
    if text.startswith('RT @'):
        retweet = True
    return {"status_id": status_id, "user": user,
            "rts": rts, "favs": favs,
            "followers": followers, "text": text,
            "retweeted": retweet}
    

# Agora, a busca pode retornar uma lista de tweets limpos
def buscar_tweets(termo):
    tweets = []
    busca = bot.search(termo)
    for tweet in busca:
        tweet_limpo = limpar_tweet(tweet)
        # Só queremos tuites originais!
        if tweet_limpo['retweeted'] == False:
            tweets.append(tweet_limpo)
    return tweets
    

In [ ]:
# Rodamos a nova função
busca = buscar_tweets('#CodaBr18')
busca

In [ ]:
# visualizando melhor a busca com pandas
import pandas as pd
df = pd.DataFrame(busca).sort_values('rts', ascending=False)
df

In [ ]:
# Reordenando a lista usando pandas e devolvendo uma lista de dicionários
df.to_dict(orient='rts')

# Você pode exportar a busca:
df.to_csv('busca_twitter.csv')

# outra maneira de fazer
from operator import itemgetter 
busca_ordenada = sorted(busca, key=itemgetter('favs'), reverse=True)

In [ ]:
# Agora selecionamos o tweet com mais engajamento:
candidato = busca_ordenada[0]

In [ ]:
# O que vamos falar pra ele?
mensagens_possiveis = ["Legal isso. Mas você viu o workshop do @burgos? Tem o código aqui: https://github.com/BurgosNY/coda-twitterbot",
                       "Essa palestra foi quase tão legal quanto a do @burgos. Olha só: https://github.com/BurgosNY/coda-twitterbot"]


In [ ]:
# Criamos uma função de enviar tweets:
def enviar_tweet(busca, mensagens_possiveis):
    busca_ordenada = sorted(busca, key=itemgetter('rts'), reverse=True)
    candidato = busca_ordenada[0]
    user = candidato['user']
    msg = random.choice(mensagens_possiveis)
    text = f'@{user} {msg}' 
    bot.update_status(text, in_reply_to_status_id=candidato['status_id'])
    print('Tweet enviado!')

In [ ]:
# Agora, tudo junto:
def twitterbot():
    termo = input('O que você busca? >')
    busca = buscar_tweets(termo)
    opcao1 = input('O que você quer tuitar? >')
    opcao2 = input('Dê outra opção >')
    enviar_tweet(busca, [opcao1, opcao2])

In [ ]:
# Testando:
twitterbot()